In [24]:
from keras.datasets import cifar10
from art.attacks import FastGradientMethod,ProjectedGradientDescent
from art.classifiers import KerasClassifier
from art.utils import load_dataset
from keras.optimizers import adam
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.models import Sequential,Model,load_model
from keras import applications
from keras.optimizers import SGD, Adam
import numpy as np
print(keras.__version__)

2.2.4-tf


In [2]:
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset(str('cifar10'))
print(y_train[0])

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


In [3]:
x_train, y_train = x_train[:500], y_train[:500]
x_test, y_test = x_test[:500], y_test[:500]
im_shape = x_train[0].shape
print(x_train.shape)

(500, 32, 32, 3)


In [4]:
base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (32,32,3))

Instructions for updating:
Colocations handled automatically by placer.


/home/xun/.local/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [5]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(10, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [6]:
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# model.fit(x_train,y_train,epochs=1, batch_size=1000, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
500/500 [==============================] - 49s 99ms/step - loss: 5.9806 - accuracy: 0.0900


In [36]:
classifier = KerasClassifier(model=model, clip_values=(min_, max_))

In [37]:
classifier.fit(x_train, y_train, nb_epochs=1, batch_size=50, verbose=1)

Epoch 1/1
10/10 [==============================] - 46s 5s/step - loss: 5.1867 - accuracy: 0.1280


In [38]:
preds = np.argmax(classifier.predict(x_train), axis=1)
print(preds[0])
acc = np.sum(preds == np.argmax(y_train)) / y_train.shape[0]
print(acc)

0
0.0


/home/xun/Documents/AdvancedDeepLearning


In [ ]:
predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on benign test examples: {}%'.format(accuracy * 100))

# Step 6: Generate adversarial test examples
attack = FastGradientMethod(classifier=classifier, eps=0.2)
x_test_adv = attack.generate(x=x_test)

# Step 7: Evaluate the ART classifier on adversarial test examples

predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on adversarial test examples: {}%'.format(accuracy * 100))

In [ ]:
# following this tutorial https://github.com/IBM/adversarial-robustness-toolbox/blob/master/examples/adversarial_training_cifar10.py

In [ ]:
import os
path = os.getcwd()
print(path)
classifier.save(filename="testing", path=path)

from keras.models import load_model
model2 = load_model(path+"/testing")
classifier2 = KerasClassifier(model=model2, clip_values=(min_, max_))

preds = np.argmax(classifier2.predict(x_train), axis=1)
print(preds[0])
acc = np.sum(preds == np.argmax(y_train)) / y_train.shape[0]
print(acc)